In [3]:
import yaml
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import subprocess
import joblib
from itertools import product

In [4]:
# Load DVC params
with open("params.yaml") as f:
    params = yaml.safe_load(f)

train_path = params["data"]["train"]
val_path = params["data"]["val"]
test_path = params["data"]["test"]

MLFLOW_URI = params["mlflow"]["uri"]
EXPERIMENT_NAME = params["mlflow"]["experiment"]

n_estimators_list = params["model"]["n_estimators_list"]
max_depth_list = params["model"]["max_depth_list"]

In [5]:
# Load processed data
train = pd.read_csv(train_path)
val = pd.read_csv(val_path)
test = pd.read_csv(test_path)

X_train, y_train = train.drop("y", axis=1), train["y"]
X_val, y_val = val.drop("y", axis=1), val["y"]
X_test, y_test = test.drop("y", axis=1), test["y"]

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/train.csv'

In [ ]:
# Remote MLflow server
mlflow.set_tracking_uri(MLFLOW_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

# Get data version (git commit hash)
data_version = subprocess.getoutput("git rev-parse HEAD")
print(f"Using data version: {data_version}")

In [ ]:
# Run experiments
for n_estimators, max_depth in product(n_estimators_list, max_depth_list):
    with mlflow.start_run():
        # Log hyperparameters and data version
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("data_version", data_version)

        # Train model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Validation metrics
        y_val_pred = model.predict(X_val)
        val_acc = accuracy_score(y_val, y_val_pred)
        val_f1 = f1_score(y_val, y_val_pred)
        val_prec = precision_score(y_val, y_val_pred)
        val_rec = recall_score(y_val, y_val_pred)

        mlflow.log_metric("val_accuracy", val_acc)
        mlflow.log_metric("val_f1", val_f1)
        mlflow.log_metric("val_precision", val_prec)
        mlflow.log_metric("val_recall", val_rec)

        # Test metrics
        y_test_pred = model.predict(X_test)
        mlflow.log_metric("test_accuracy", accuracy_score(y_test, y_test_pred))
        mlflow.log_metric("test_f1", f1_score(y_test, y_test_pred))
        mlflow.log_metric("test_precision", precision_score(y_test, y_test_pred))
        mlflow.log_metric("test_recall", recall_score(y_test, y_test_pred))

        # Log model
        mlflow.sklearn.log_model(model, "model")

        print(f"✅ Run logged: n_estimators={n_estimators}, max_depth={max_depth}, val_f1={val_f1:.4f}")